In [ ]:
# GENERATE TEXT EMBEDDINGS IN BATCHES OF 128

import faiss
import numpy as np
import pandas as pd
import requests
import os

# Define Paths
FAISS_TEXT_INDEX_PATH = "Embeddings/text-embeddings/faiss_index.bin"
EMBEDDINGS_CSV_PATH = "Embeddings/text-embeddings/embeddings.csv"

# Define Runpod API Endpoint
RUNPOD_API_ENDPOINT = "https://26oe7qh881svy7-5000.proxy.runpod.net/"

df_products = pd.read_csv("dataset/textual/complete/final_products.csv")

# Generate text inputs
text_inputs = df_products.apply(
    lambda row: f"Title of Product: {row['title']}\nProduct Image Description: {row['llava_generated_image_caption']}\nProduct Category: {row['category_name']}", 
    axis=1
).tolist()

# Get Text Embeddings form NVIDIA Triton Inference Server
def get_embeddings(text_inputs, batch_size=128):
    embeddings = []
    for i in range(0, len(text_inputs), batch_size):
        batch = text_inputs[i:i + batch_size]
        response = requests.post(f"{RUNPOD_API_ENDPOINT}/infer_text", json=batch)
        response_json = response.json()
        batch_embeddings = np.array(response_json['result'])  # Shape (batch_size, tokens, 1024)
        batch_embeddings = np.mean(batch_embeddings, axis=1)  # Averaging tokens to get single vector per text
        embeddings.append(batch_embeddings)
    return np.vstack(embeddings)

# Check if FAISS index exists
if os.path.exists(FAISS_TEXT_INDEX_PATH):
    print("Loading existing FAISS index...")
    index = faiss.read_index(FAISS_TEXT_INDEX_PATH)
    embeddings = np.loadtxt(EMBEDDINGS_CSV_PATH, delimiter=",")  # Load saved embeddings
else:
    print("Generating embeddings and saving FAISS index...")
    embeddings = get_embeddings(text_inputs)

    # Create FAISS Index
    d = embeddings.shape[1]
    index = faiss.IndexFlatL2(d)
    index.add(embeddings)

    # Save FAISS index
    faiss.write_index(index, FAISS_TEXT_INDEX_PATH)
    np.savetxt(EMBEDDINGS_CSV_PATH, embeddings, delimiter=",")  # Save embeddings

print(f"FAISS index contains {index.ntotal} vectors.")


🔹 Generating embeddings and saving FAISS index...
✅ FAISS index contains 10200 vectors.


In [ ]:
# Function to find similar products using input embedding and FAISS index

def find_similar(query_text, index, df_products, k=5):
    response = requests.post(f"{RUNPOD_API_ENDPOINT}/infer_text", json=[query_text])
    query_embedding = np.array(response.json()['result'])
    query_embedding = np.mean(query_embedding, axis=1)  # Average token embeddings for the single query text

    distances, indices = index.search(query_embedding, k)  # Search FAISS index

    # Retrieve the top similar products
    similar_products = df_products.iloc[indices[0]].copy()
    similar_products['distance'] = distances[0]  # Attach similarity scores
    
    return similar_products

# Example Query
query_text = "Macbook pro"
index = faiss.read_index(FAISS_TEXT_INDEX_PATH)
df_products = pd.read_csv("dataset/textual/complete/final_products.csv")
top_results = find_similar(query_text, index, df_products)

# Display Results
print("\n🔍 Top 5 Similar Products:\n")
print(top_results[['title', 'category_name', 'distance']])  # Show key details


🔍 Top 5 Similar Products:

                                                 title       category_name  \
244  for MacBook Pro 13 inch Case M2/M1 A2338 A2289...  Laptop Accessories   
528  for MacBook Pro 14 Inch 2023 2022 2021 Case M2...  Laptop Accessories   
636  IBENZER Compatible MacBook Pro 14 Inch Case 20...  Laptop Accessories   
548  A1398 Macbook Pro Battery A1417 for Macbook pr...  Laptop Accessories   
330  Compatible with MacBook Pro 13 inch Case M2 20...  Laptop Accessories   

       distance  
244  161.664963  
528  162.194870  
636  162.937637  
548  163.861755  
330  166.678452  


In [ ]:
# # GENERATE IMAGE EMBEDDINGS IN BATCHES OF 64

import faiss
import numpy as np
import pandas as pd
import requests
import os
from io import BytesIO

# Define Paths for Image FAISS Index
FAISS_INDEX_IMG_PATH = "Embeddings/image_embeddings/faiss_index_images.bin"
IMAGE_EMBEDDINGS_CSV_PATH = "Embeddings/image_embeddings/image_embeddings.csv"

# Define Runpod API Endpoint
RUNPOD_API_ENDPOINT = "https://26oe7qh881svy7-5000.proxy.runpod.net/"

df_products = pd.read_csv("dataset/textual/complete/final_products.csv")

image_urls = df_products['imgUrl'].tolist()

# Function to Download Images and Get Embeddings
def get_image_embeddings(image_urls, batch_size=64):
    embeddings = []
    
    for i in range(0, len(image_urls), batch_size):
        batch_urls = image_urls[i:i + batch_size]
        files = []

        # Download images
        for j, url in enumerate(batch_urls):
            response = requests.get(url)
            if response.status_code == 200:
                image_bytes = BytesIO(response.content)
                files.append(("images", (f"image_{i+j}.jpg", image_bytes, "image/jpeg")))

        # Send request to NVIDIA Triton Inference Server
        response = requests.post(f"{RUNPOD_API_ENDPOINT}/infer_image", files=files)
        response_json = response.json()
        batch_embeddings = np.array(response_json['image_embeddings'])  # (batch_size, 768) for this model
        embeddings.append(batch_embeddings)

    return np.vstack(embeddings)  # Combine all batches

if os.path.exists(FAISS_INDEX_IMG_PATH) and os.path.exists(IMAGE_EMBEDDINGS_CSV_PATH):
    print("Loading existing FAISS index for images...")
    index_images = faiss.read_index(FAISS_INDEX_IMG_PATH)
    image_embeddings = np.loadtxt(IMAGE_EMBEDDINGS_CSV_PATH, delimiter=",")
else:
    print("Generating image embeddings and saving FAISS index...")
    image_embeddings = get_image_embeddings(image_urls)

    # Create FAISS Index for images
    d_img = image_embeddings.shape[1]  # Should be 768 dimensions for this model
    index_images = faiss.IndexFlatL2(d_img)
    index_images.add(image_embeddings)

    # Save FAISS index
    faiss.write_index(index_images, FAISS_INDEX_IMG_PATH)
    np.savetxt(IMAGE_EMBEDDINGS_CSV_PATH, image_embeddings, delimiter=",")

print(f"FAISS index contains {index_images.ntotal} image vectors.")

🔹 Generating image embeddings and saving FAISS index...
✅ FAISS index contains 10199 image vectors.


In [ ]:
# Function to find similar products using image embeddings generated by a URL and FAISS index

def find_similar_images(image_url, k=5):
    response = requests.get(image_url)
    if response.status_code != 200:
        raise ValueError("Failed to download image!")

    image_bytes = BytesIO(response.content)
    files = [("images", ("query_image.jpg", image_bytes, "image/jpeg"))]

    # Get query image embedding
    response = requests.post(f"{RUNPOD_API_ENDPOINT}/infer_image", files=files)
    query_embedding = np.array(response.json()['image_embeddings'])  # (1, 768)

    # Search FAISS index
    distances, indices = index_images.search(query_embedding, k)

    similar_images = df_products.iloc[indices[0]].copy()
    similar_images['distance'] = distances[0]

    return similar_images

# Example Image Query
query_image_url = "https://m.media-amazon.com/images/I/710N2S69NvL._AC_UL320_.jpg"
top_image_results = find_similar_images(query_image_url)

print("Top 5 Similar Images:\n")
print(top_image_results[['title', 'imgUrl', 'distance']])  # Show relevant details


In [ ]:
# An optional function to wait for the Triton server to be ready before making requests
import requests
import time

RUNPOD_API_ENDPOINT = "https://lhr3tmm415zpim-5000.proxy.runpod.net/"

def wait_for_server():
    for _ in range(10):
        try:
            response = requests.get(f"{RUNPOD_API_ENDPOINT}/health")
            if response.status_code == 200 and response.json().get("status") == "ok":
                print("Triton is ready!")
                return True
        except requests.exceptions.RequestException:
            pass
        
        print("Waiting for Triton server to be ready...")
        time.sleep(3)
    
    print("Triton server is not available after multiple attempts.")
    return False

wait_for_server()

✅ Triton is ready!


True